In [1]:
import re
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm

In [2]:
# Version information
month = '2021'
version = 'v1'
variable_type = 'main'
source = 'scopus'

def filename_gen (path, filename, extension):
    if variable_type != 'main':
        return path + source + '/' + '_'.join([filename, month, version],) + variable_type + extension
    else:
        return path + source + '/' + '_'.join([filename, month, version],) + extension

# META
ORG_TYPE_UNIVERSITY = 'university'
ORG_TYPE_INSTITUTE = 'institute'
ORG_TYPE_FIRM = 'firm'

In [3]:
main_df = pd.read_csv('../../data/scopus/final_2021.csv', low_memory=False)

alias_dict = dict()

csRankingUnivs = set(pd.read_csv('../../data/organizations/top_programs_univ.csv')['Alias'])
fortuneGlobal500Tech = set(pd.read_csv('../../data/organizations/top_programs_firms.csv')['Alias'])
topInstitutes = set(pd.read_csv('../../data/organizations/top_programs_institutes.csv')['Alias'])
qs_2011_df = pd.read_csv('../../data/organizations/qs_rank_2011.csv')

#Inc, LTD,LLC,Consult,Telecom,Industries,LLP,GmbH,semiconductor,company,Technologies,Inc,Incorporated 
other_firms_keywords = ['ltd', 'llc', 'inc', 'limited', 'consult', 'industries', 'llp', 'gmbh',
                       'corp', 'incorporated', 'incorporation', 'corporation', 'company']

univ_keywords_starts_with = ['univ', 'college', 'dept', 'faculty', 'school', 'polytechnic']
inst_keywords_starts_with = ['inst', 'govt', 'defense',  'ministry', 'hospital', 'military', 'defence']

In [4]:
fuzzy_dict = dict()

In [5]:
cnt = 0
universities = pd.read_csv('../../data/organizations/univ_ranking_dec_v2.csv')
num_universities = len(universities)
for university_iter in range(0, num_universities):
    univ_name = universities.at[university_iter, 'name']
    alias_dict[univ_name] = dict()
    alias_dict[univ_name]['type'] = ORG_TYPE_UNIVERSITY
    alias_dict[univ_name]['aliases'] = universities.at[university_iter, 'aliases'].split(';')
    alias_dict[univ_name]['exclude_aliases'] = universities.at[university_iter, 'exclude']
    alias_dict[univ_name]['exact_match'] = universities.at[university_iter, 'exact_match'] 
    alias_dict[univ_name]['regEx'] = universities.at[university_iter, 'regEx']
    alias_dict[univ_name]['usnews'] = universities.at[university_iter, 'usnews']
    alias_dict[univ_name]['qs'] = universities.at[university_iter, 'qs']
    alias_dict[univ_name]['csRankingUnivs'] = univ_name in csRankingUnivs
    if len(qs_2011_df[qs_2011_df['alias'] == univ_name]['qs_rank_2011']) > 0:
        alias_dict[univ_name]['qs_2011'] = qs_2011_df[qs_2011_df['alias'] == univ_name]['qs_rank_2011'].values[0]
        cnt+=1
        
del university_iter
del universities
del num_universities
del univ_name

organizations = pd.read_csv('../../data/organizations/organizations_dec_v2.csv')
num_organizations = len(organizations)
for organization_iter in range(0, num_organizations):
    org_name = organizations.at[organization_iter, 'name']
    alias_dict[org_name] = dict()
    alias_dict[org_name]['type'] = organizations.at[organization_iter, 'type']
    if pd.isna(organizations.at[organization_iter, 'alias']) == False:
        alias_dict[org_name]['aliases'] = organizations.at[organization_iter, 'alias'].split(';')
    else:
        alias_dict[org_name]['aliases'] = []
    alias_dict[org_name]['exclude_aliases'] = organizations.at[organization_iter, 'exclude']
    alias_dict[org_name]['exact_match'] = organizations.at[organization_iter, 'exact_match']
    alias_dict[org_name]['regEx'] = organizations.at[organization_iter, 'regEx']
    if alias_dict[org_name]['type'] == ORG_TYPE_UNIVERSITY:
        alias_dict[org_name]['usnews'] = None
        alias_dict[org_name]['qs'] = None
        alias_dict[org_name]['csRankingUnivs'] = org_name in csRankingUnivs
        if len(qs_2011_df[qs_2011_df['alias'] == org_name]['qs_rank_2011']) > 0:
            cnt+=1
            alias_dict[org_name]['qs_2011'] = qs_2011_df[qs_2011_df['alias'] == org_name]['qs_rank_2011'].values[0]
    if alias_dict[org_name]['type'] == ORG_TYPE_FIRM:
        alias_dict[org_name]['fortuneGlobal500Tech'] = org_name in fortuneGlobal500Tech
    if alias_dict[org_name]['type'] == ORG_TYPE_INSTITUTE:
        alias_dict[org_name]['topInstitutes'] = org_name in topInstitutes
del num_organizations
del organization_iter
del organizations
del org_name

print(cnt)
organizations_name = list(alias_dict.keys())

499


In [6]:
replace_strings = {
    ' technology ': ' tech ',
    ' technology': ' tech',
    'technology ': 'tech ',
    'technische': 'technical',
    ' university ': ' univ ',
    ' university': ' univ',
    'universiteit': 'univ',
    'university ': 'univ ',
    'universität': 'univ',
    'universitat': 'univ',
    'universitaet': 'univ',
    'université': 'univ',
    'università': 'univ',
    'universita': 'univ',
    'universidade': 'univ',
    'universidad': 'univ',
    'universiti': 'univ',
    'universite': 'univ',
    'univs': 'univ',
    ' institute ': ' inst ',
    ' institute': ' inst',
    'institute ': 'inst ',
    'technical univ': 'tu',
    ' of ': ' ',
    ' at ': ' ',
    ' the ': ' ',
    'the ': ' ',
    ' and ': ' & ',
    '&': ' & ',
    'national': 'natl',
    'corporations': 'corp',
    'corporation': 'corp',
    'laboratories': 'labs',
    'laboratory': 'lab',
    'department': 'dept',
    'government': 'govt',
    '(': ' ',
    ')': ' ',
}
print(replace_strings)

{' technology ': ' tech ', ' technology': ' tech', 'technology ': 'tech ', 'technische': 'technical', ' university ': ' univ ', ' university': ' univ', 'universiteit': 'univ', 'university ': 'univ ', 'universität': 'univ', 'universitat': 'univ', 'universitaet': 'univ', 'université': 'univ', 'università': 'univ', 'universita': 'univ', 'universidade': 'univ', 'universidad': 'univ', 'universiti': 'univ', 'universite': 'univ', 'univs': 'univ', ' institute ': ' inst ', ' institute': ' inst', 'institute ': 'inst ', 'technical univ': 'tu', ' of ': ' ', ' at ': ' ', ' the ': ' ', 'the ': ' ', ' and ': ' & ', '&': ' & ', 'national': 'natl', 'corporations': 'corp', 'corporation': 'corp', 'laboratories': 'labs', 'laboratory': 'lab', 'department': 'dept', 'government': 'govt', '(': ' ', ')': ' '}


In [7]:
def text_transform (text_raw):
    re.sub(r"^U", 'univ', text_raw)
    re.sub(r" U", ' univ', text_raw)
    
    text_raw = text_raw.strip()
    text_raw = text_raw.lower()
    for to_replace in replace_strings.keys():
        text_raw = text_raw.replace(to_replace, replace_strings[to_replace]) 
    
    text_raw = re.sub(r"[,-]+\ *", " ", text_raw)
    text_raw = re.sub(r"[/]", " ", text_raw)
    text_raw = re.sub(r"[.]", "", text_raw) # M.I.T > MIT
    text_raw = re.sub(r'[()]', ' ', text_raw)
    text_raw = re.sub(r'\s+', ' ', text_raw)
    
    text_raw = text_raw.strip()
    return text_raw

def text_transform_conservative (text_raw):
    text_raw = text_raw.strip()
    
    text_raw = re.sub(r"[,-]+\ *", " ", text_raw)
    text_raw = re.sub(r"[/]", " ", text_raw)
    text_raw = re.sub(r"[.]", "", text_raw) # M.I.T > MIT
    text_raw = re.sub(r'[()]', ' ', text_raw)
    text_raw = re.sub(r'\s+', ' ', text_raw)
    
    text_raw = text_raw.strip()
    return text_raw

In [8]:
def contains (needle, haystack):
    if len(needle) == 0:
        return False
    haystack = ' ' + haystack + ' '
    needle = ' ' + needle + ' '
    return needle in haystack

In [9]:
def search_with_aliases_direct (aliases, affiliation_text):
    for alias in aliases:
        if (contains(text_transform(alias), affiliation_text)):
            return True
    return False

def org_alias_search (org_name, affiliation_text):
    aliases = alias_dict[org_name]['aliases']
    return search_with_aliases_direct(aliases, affiliation_text)

def exact_search_with_aliases_direct (aliases, affiliation_text):
    for alias in aliases:
        if (contains(alias, affiliation_text)):
            return True
    return False

def regex_search_with_aliases_direct (regex_aliases, affiliation_text):
    for alias in regex_aliases:
        alias_words = alias.split(' ')
        affiliation_text_words = affiliation_text.split(' ')
        if alias_words[0] not in affiliation_text_words:
            continue
        alias_words_iter = 0
        for affiliation_text_word in affiliation_text_words:
            if alias_words[alias_words_iter] == affiliation_text_word:
                alias_words_iter += 1
                if alias_words_iter == len(alias_words):
                    return True
        
    return False

In [10]:
def get_fuzzy_ratio (needle, haystack):
    if len(needle) == 0:
        return False
    haystack = ' ' + haystack + ' '
    needle = ' ' + needle + ' '
    return fuzz.partial_ratio(haystack, needle)

In [11]:
def org_alias_fuzzy_search (org_name, affiliation_text):
    aliases = alias_dict[org_name]['aliases']
    ret = []
    for alias in aliases:
        if len(alias.split(' ')) == 1:
            continue
        ret.append((alias, get_fuzzy_ratio(alias, affiliation_text)))
    if len(ret):
        ret = sorted(ret, key=lambda x: x[1], reverse=True)
    if len(ret) and ret[0][1] > 90:
        return [True, ret]
    else:
        return [False, ret]
def find_other_firms (affiliation_text):
    for other_firm_keyword in other_firms_keywords:
        if contains(other_firm_keyword, affiliation_text):
            return True
    return False

def is_other_univ (affiliation_text):
    for st_kw in univ_keywords_starts_with:
        if st_kw not in affiliation_text:
            continue
        words = affiliation_text.split(' ')
        for word in words:
            if word.startswith(st_kw):
                return True
    return False

def is_other_inst (affiliation_text):
    for st_kw in inst_keywords_starts_with:
        if st_kw not in affiliation_text:
            continue
        words = affiliation_text.split(' ')
        for word in words:
            if word.startswith(st_kw):
                return True
    return False

In [12]:
def find_affiliations_single (affiliation_text_raw):
    univ, inst, firm = None, None, None
    
    for org_name in organizations_name:
        current_org_dict = alias_dict[org_name]
        dontCheck = False
        isFound = False
        
        if univ != None and current_org_dict['type'] == ORG_TYPE_UNIVERSITY:
            continue
        if inst != None and current_org_dict['type'] == ORG_TYPE_INSTITUTE:
            continue
        if firm != None and current_org_dict['type'] == ORG_TYPE_FIRM:
            continue
            
        exact_match_aliases = current_org_dict['exact_match']
        if pd.isna(exact_match_aliases) == False:
            exact_match_aliases = exact_match_aliases.split(';')
            affiliation_text_for_exact_match = text_transform_conservative(affiliation_text_raw)
            if exact_search_with_aliases_direct(exact_match_aliases, affiliation_text_for_exact_match):
                isFound = True
        
        affiliation_text = text_transform(affiliation_text_raw)
    
        if isFound == False and pd.isna(current_org_dict['exclude_aliases']) == False:
            exclude_aliases = current_org_dict['exclude_aliases'].split(';')
            if search_with_aliases_direct(exclude_aliases, affiliation_text):
                dontCheck = True
        
        if dontCheck:
            continue
        
        if isFound == False and org_alias_search(org_name, affiliation_text):
            isFound = True
        
        if isFound == False and pd.isna(current_org_dict['regEx']) == False:
            regex_aliases = current_org_dict['regEx'].split(';')
            if regex_search_with_aliases_direct(regex_aliases, affiliation_text):
                isFound = True
    
        if isFound:
            if current_org_dict['type'] == ORG_TYPE_UNIVERSITY:
                univ = org_name
            if current_org_dict['type'] == ORG_TYPE_INSTITUTE:
                inst = org_name
            if current_org_dict['type'] == ORG_TYPE_FIRM:
                firm = org_name
    
    if univ == None and inst == None and firm == None:
        for org_name in organizations_name:
            current_org_dict = alias_dict[org_name]
            dontCheck = False
            isFound = False

            affiliation_text = text_transform(affiliation_text)

            if isFound == False and pd.isna(current_org_dict['exclude_aliases']) == False:
                exclude_aliases = current_org_dict['exclude_aliases'].split(';')
                if search_with_aliases_direct(exclude_aliases, affiliation_text):
                    dontCheck = True

            if dontCheck:
                continue

            if isFound == False and org_alias_fuzzy_search(org_name, affiliation_text)[0]:
                isFound = True
                
                if affiliation_text not in fuzzy_dict.keys():
                    fuzzy_dict[affiliation_text] = {
                        'raw': affiliation_text_raw,
                        'name': affiliation_text,
                        'match': org_name,
                        'count': 0
                    }
                fuzzy_dict[affiliation_text]['count'] += 1

            if isFound:
                if current_org_dict['type'] == ORG_TYPE_UNIVERSITY:
                    univ = org_name
                if current_org_dict['type'] == ORG_TYPE_INSTITUTE:
                    inst = org_name
                if current_org_dict['type'] == ORG_TYPE_FIRM:
                    firm = org_name
    
    return univ, inst, firm

In [13]:
uniq_affiliations = dict()

total_papers = len(main_df)
for df_iter in tqdm(range(0, total_papers)):
    affiliations = main_df.loc[df_iter]['Affiliations']
    if pd.isna(affiliations):
        continue
    affiliations = affiliations.split(';')
    
    for affiliation in affiliations:
        if pd.isna(affiliation) or affiliation == '':
            continue
        affiliation_text_raw = affiliation
        affiliation = text_transform(affiliation)
        if affiliation not in uniq_affiliations.keys():
            uniq_affiliations[affiliation] = dict()
            uniq_affiliations[affiliation]['raw'] = affiliation_text_raw
            
            uniq_affiliations[affiliation]['count'] = 0
            
            uniq_affiliations[affiliation]['selected_university'] = None
            uniq_affiliations[affiliation]['selected_institute'] = None
            uniq_affiliations[affiliation]['selected_firm'] = None
            uniq_affiliations[affiliation]['other_universities'] = None
            uniq_affiliations[affiliation]['other_institutes'] = None
            uniq_affiliations[affiliation]['other_firms'] = None
            uniq_affiliations[affiliation]['none'] = False
            
            uniq_affiliations[affiliation]['usnews'] = None
            uniq_affiliations[affiliation]['qs'] = None
            uniq_affiliations[affiliation]['csRankingUnivs'] = None
            uniq_affiliations[affiliation]['topInstitutes'] = None
            uniq_affiliations[affiliation]['fortuneGlobal500Tech'] = None
            
            univ, inst, firm = find_affiliations_single(affiliation_text_raw)
            if univ != None:
                uniq_affiliations[affiliation]['selected_university'] = univ
                univ_dict = alias_dict[univ]
                uniq_affiliations[affiliation]['usnews'] = univ_dict['usnews']
                uniq_affiliations[affiliation]['qs'] = univ_dict['qs']
                uniq_affiliations[affiliation]['csRankingUnivs'] = univ_dict['csRankingUnivs']
                if 'qs_2011' in univ_dict:
                    uniq_affiliations[affiliation]['qs_2011'] = univ_dict['qs_2011']
            if inst != None:
                uniq_affiliations[affiliation]['selected_institute'] = inst
                inst_dict = alias_dict[inst]
                uniq_affiliations[affiliation]['topInstitutes'] = inst_dict['topInstitutes']
            if firm != None:
                uniq_affiliations[affiliation]['selected_firm'] = firm
                firm_dict = alias_dict[firm]
                uniq_affiliations[affiliation]['fortuneGlobal500Tech'] = firm_dict['fortuneGlobal500Tech']
            if univ == None and inst == None and firm == None:
                uniq_affiliations[affiliation]['none'] = True
            if firm == None and find_other_firms(affiliation):
                uniq_affiliations[affiliation]['other_firms'] = True
            if univ == None and is_other_univ(affiliation):
                uniq_affiliations[affiliation]['other_universities'] = True
            if inst == None and is_other_inst(affiliation):
                uniq_affiliations[affiliation]['other_institutes'] = True
            
        uniq_affiliations[affiliation]['count'] += 1

100%|██████████| 186880/186880 [12:46:48<00:00,  4.06it/s]        


In [14]:
new_df = pd.DataFrame(uniq_affiliations).transpose().reset_index()
new_df = new_df.rename(columns={'index': 'name'})
new_df.to_csv(filename_gen('../../data/', 'aff_univ_inst_mapping', '.csv'), index=None)

In [15]:
fuzzy_df = pd.DataFrame(fuzzy_dict).transpose().sort_values(by='count', ascending=False)
fuzzy_df.to_csv('../../data/scopus/fuzzy.csv')